# Enviar formulários atualizados para o banco de dados

Deve-se executar as duas primeiras células para carregar funções utilizadas para rodar o script e verificar se está em Produção

In [1]:
import json
import os
from openpyxl import load_workbook
from datetime import timedelta
import random
from copy import deepcopy
from tqdm.notebook import tqdm

In [2]:
!cat .env

STAGE=PRODUCTION


Criando diretório com nome do arquivo

In [3]:
!mkdir "Elo Eco Noroeste - Parametrização Unidade"

Movendo arquivo .zip para o diretório

In [4]:
!mv "Elo Eco Noroeste - Parametrização Unidade.zip" "Elo Eco Noroeste - Parametrização Unidade"/

Descompactando arquivo .zip

In [5]:
!cd "Elo Eco Noroeste - Parametrização Unidade"/ && unzip "Elo Eco Noroeste - Parametrização Unidade.zip"

Archive:  Elo Eco Noroeste - Parametrização Unidade.zip
  inflating: 1.txt                   
  inflating: 10.txt                  
  inflating: 11.txt                  
  inflating: 12.txt                  
  inflating: 13.txt                  
  inflating: 14.txt                  
  inflating: 15.txt                  
  inflating: 16.txt                  
  inflating: 17.txt                  
  inflating: 18.txt                  
  inflating: 19.txt                  
  inflating: 2.txt                   
  inflating: 20.txt                  
  inflating: 21.txt                  
  inflating: 22.txt                  
  inflating: 23.txt                  
  inflating: 24.txt                  
  inflating: 25.txt                  
  inflating: 26.txt                  
  inflating: 27.txt                  
  inflating: 28.txt                  
  inflating: 29.txt                  
  inflating: 3.txt                   
  inflating: 30.txt                  
  inflating: 31.txt             

Necessário executar somente se estiver no MAC

In [6]:
!cd "Elo Eco Noroeste - Parametrização Unidade"/ && rm -rf __MACOSX

Deletando arquivo .zip

In [7]:
!cd "Elo Eco Noroeste - Parametrização Unidade"/ && rm "Elo Eco Noroeste - Parametrização Unidade.zip"

**Escolhendo a unidade que desejar**

In [9]:
company = Company.objects.get(name="ELO EcoNoroeste")
company

<Company: 0dcad023-ac00-4141-8262-8f1688f4b7bd: ELO EcoNoroeste>

Carregando os dados dos arquivos .txt gerados

In [10]:
new_forms = []

for index, file in enumerate(os.listdir('Elo Eco Noroeste - Parametrização Unidade/')):
    if not '.txt' in file:
        continue
    file_name = 'Elo Eco Noroeste - Parametrização Unidade/{}'.format(file)
    print('------')
    print(file_name)
    with open(file_name, 'r+') as fo:
        form = json.load(fo)
        print(form['displayName'])
        new_forms.append(form)

------
Elo Eco Noroeste - Parametrização Unidade/34.txt
Lavagem de praça de pedágios
------
Elo Eco Noroeste - Parametrização Unidade/12.txt
Juntas e trincas
------
Elo Eco Noroeste - Parametrização Unidade/7.txt
Selagem de trincas
------
Elo Eco Noroeste - Parametrização Unidade/24.txt
Lixo, resíduos, entulho ou restos vegetais
------
Elo Eco Noroeste - Parametrização Unidade/51.txt
Limpeza e manutenção de barreira de concreto
------
Elo Eco Noroeste - Parametrização Unidade/1.txt
Panela, buraco ou desplacamento
------
Elo Eco Noroeste - Parametrização Unidade/26.txt
Limpeza e varredura de áreas pavimentadas
------
Elo Eco Noroeste - Parametrização Unidade/5.txt
Pano de rolamento comprometido
------
Elo Eco Noroeste - Parametrização Unidade/14.txt
Roçada Manual
------
Elo Eco Noroeste - Parametrização Unidade/42.txt
Limpeza das drenagens dos túneis
------
Elo Eco Noroeste - Parametrização Unidade/50.txt
Limpeza e manutenção de defensa metálica
------
Elo Eco Noroeste - Parametrização 

Testando form para conferir se dados foram carregados corretamente

In [11]:
len(new_forms),new_forms[2]

(51,
 {'id': 7,
  'name': 'Selagem de trincas',
  'displayName': 'Selagem de trincas',
  'fields': [{'displayName': 'Origem',
    'apiName': 'origem',
    'dataType': 'select',
    'id': 1,
    'selectOptions': {'options': [{'name': 'Solicitação', 'value': '1'},
      {'name': 'KCOR/KRIA', 'value': '2'},
      {'name': 'Não Conformidade', 'value': '3'},
      {'name': 'Diário', 'value': '4'},
      {'name': 'Obras', 'value': '5'},
      {'name': 'Programação Mensal', 'value': '6'},
      {'name': 'Ouvidoria', 'value': '7'},
      {'name': 'Preventivo', 'value': '8'},
      {'name': 'Outro', 'value': '9'}]}},
   {'displayName': 'Ordem de Serviço',
    'apiName': 'ordemServico',
    'dataType': 'number',
    'id': 2},
   {'displayName': 'Metros',
    'apiName': 'metros',
    'dataType': 'float',
    'unit': 'm',
    'id': 3},
   {'displayName': 'Observação',
    'apiName': 'notes',
    'dataType': 'textArea',
    'id': 4}],
  'groups': [{'order': 1, 'members': [1, 2], 'displayName': 'Def

Escolhendo nome do arquivo Excel para carregar as Classes

In [12]:
filename = "Elo Eco Noroeste - Parametrização Unidade/Elo Eco Noroeste - Parametrização Unidade"

Escolhendo a aba da planilha do Excel

Obs: atenção para escolher a aba correta da planilha

In [13]:
wb = load_workbook(filename + ".xlsx")
ws = wb['Classes e Nat']
# ws = wb['Classes']

Testando planilha carregada do Excel

In [14]:
# print(ws['A1'].value)
print(ws['F1'].value)

Formulário


Carregando as classes no script

In [15]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
    values.append(obj)

Testando as configurações das Classes para conferir se dados foram carregados corretamente

In [16]:
values[1]

{None: None,
 'Formulário': 2,
 'Natureza': 'Pavimento Flexível',
 'Classe': 'Reparo definitivo com recorte - Pavimento Flexível',
 'Observação': None,
 'Inventário/Monitoração': None,
 'ID Classe': 2,
 'Existe Natureza?': 3}

Inserindo as informações de Natureza e Prazo no form

In [17]:
for a in values:
    if a['Classe'] not in [x['displayName'] for x in new_forms]:
        print(a['Classe'])

In [18]:
for form in new_forms:
    if form['displayName'] not in [a['Classe'] for a in values]:
        print(form['displayName'])
    else:
        row = next(a for a in values if a['Classe'] == form['displayName'])
        form['kind'] = row['Existe Natureza?']
#         form['kind'] = row['id_Natureza']

        if row.get('Prazo'):
            if 'horas' in row['Prazo']:
                form['deadline'] = timedelta(hours=int(row['Prazo'].split(' ')[0]))
            elif 'dias' or 'dia' in row['Prazo']:
                form['deadline'] = timedelta(days=int(row['Prazo'].split(' ')[0]))
            else:
                print('DEU RUIM')
        if row.get('Cor'):
            form['color'] = row['Cor']

Testando o form modificado

In [19]:
for a in new_forms:
    a['name'] = a['name'].strip()
    a['displayName'] = a['displayName'].strip()

In [20]:
new_forms[8]

{'id': 14,
 'name': 'Roçada Manual',
 'displayName': 'Roçada Manual',
 'fields': [{'displayName': 'Origem',
   'apiName': 'origem',
   'dataType': 'select',
   'id': 1,
   'selectOptions': {'options': [{'name': 'Solicitação', 'value': '1'},
     {'name': 'KCOR/KRIA', 'value': '2'},
     {'name': 'Não Conformidade', 'value': '3'},
     {'name': 'Diário', 'value': '4'},
     {'name': 'Obras', 'value': '5'},
     {'name': 'Programação Mensal', 'value': '6'},
     {'name': 'Ouvidoria', 'value': '7'},
     {'name': 'Preventivo', 'value': '8'},
     {'name': 'Outro', 'value': '9'}]}},
  {'displayName': 'Ordem de Serviço',
   'apiName': 'ordemServico',
   'dataType': 'number',
   'id': 2},
  {'displayName': 'Largura',
   'apiName': 'width',
   'dataType': 'float',
   'unit': 'm',
   'id': 3},
  {'displayName': 'Comprimento',
   'apiName': 'length',
   'dataType': 'float',
   'unit': 'm',
   'id': 4},
  {'displayName': 'Área',
   'apiName': 'area',
   'dataType': 'float',
   'autofill': {'if':

Salvando as modificações dos forms nos OccurrenceTypes já criados no sistema

## Atenção! Este passo modifica informações do banco de dados

In [21]:
counter=0
occurrence_types = []

for form in tqdm(new_forms):
    form_copy = deepcopy(form)
    if 'deadline' in form_copy.keys():
        del form_copy['deadline']
    if 'kind' in form_copy.keys():
        del form_copy['kind']
    if 'color' in form_copy.keys():
        del form_copy['color']
    
#     if form['displayName'] in [x.name for x in OccurrenceType.objects.filter(company=company)] and if str(form['kind']) in [x.occurrence_kind for x in OccurrenceType.objects.filter(name=form['displayName'])]:
    if form['displayName'] in [x.name for x in OccurrenceType.objects.filter(company=company)]:

        occurrence_type=OccurrenceType.objects.filter(company=company, name=form['displayName']).first()
        occurrence_type.occurrence_kind=form['kind']
        occurrence_type.form_fields=form_copy
        occurrence_type.deadline=form['deadline'] if 'deadline' in form else None
        occurrence_type.save()
        print("Atualizei o formulário: ", occurrence_type.name)
#         counter+=1
    else:
        occurrence_type=OccurrenceType(name=form['displayName'], occurrence_kind=form['kind'], 
            form_fields=form_copy, deadline=form['deadline'] if 'deadline' in form else None)
        print(occurrence_type)
        occurrence_type.save()
        spec=OccurrenceTypeSpecs(occurrence_type=occurrence_type, company=company,
            color=form['color'] if 'color' in form else "#"+''.join(random.choices('0123456789ABCDEF', k=6)))
        spec.save()
        counter+=1

print(counter)

  0%|          | 0/51 [00:00<?, ?it/s]

Lavagem de praça de pedágios - []
Juntas e trincas - []
Selagem de trincas - []
Lixo, resíduos, entulho ou restos vegetais - []
Limpeza e manutenção de barreira de concreto - []
Panela, buraco ou desplacamento - []
Limpeza e varredura de áreas pavimentadas - []
Pano de rolamento comprometido - []
Roçada Manual - []
Limpeza das drenagens dos túneis - []
Limpeza e manutenção de defensa metálica - []
Lavagem de balizadores - []
Varrição de Áreas Pavimentadas, Capina e Limpeza - []
Degrau entre pista e acostamento - []
Limpeza de drenagem profunda - []
Depressão no encontro com OAE - Pavimento Flexível - []
Manutenção de Árvores e Arbustos - []
Limpeza de Dispositivos de Drenagem Internos e Externos - []
Lavagem de delineadores - []
Depressão no encontro com OAE - Pavimento Rígido - []
Limpeza de sinalização Vertical de Solo - []
Limpeza de Drenagem Superficial de Plataforma - []
Instalação de defensa metálica - []
Roçada Mecanizada - []
Panela ou buraco - []
Animais mortos - []
Lavagem de

## Linhas para testes (não considerar para execução do script)

  0%|          | 0/102 [00:00<?, ?it/s]

0

# Criar apartir de uma classe existente

In [ ]:
company=Company.objects.get(name='Eixo SP')
occ=OccurrenceType.objects.get(company__name='Eixo SP',name='Manutenção de junta de dilatação')
occ,company

In [ ]:
#Nomes das classes novas
lista=['Totem não funciona',
'Iluminação interna queimada',
'Iluminação externa queimada',
'Tomada externa com defeito',
'Tomada interna com defeito',]
len(lista)

In [ ]:
from tqdm.notebook import tqdm
for a in tqdm(lista):
    new_occ=occ
    new_occ.uuid=None
    new_occ.name=a.strip()
    new_occ.occurrence_kind='23'
    new_occ.form_fields['name'] = a.strip()
    new_occ.form_fields['displayName'] = a.strip()
    new_occ.save()
    
    spec=OccurrenceTypeSpecs(occurrence_type=new_occ, company=company)
    spec.save()

In [43]:

for form in tqdm(new_forms[9:]):
    try:
        occ = OccurrenceType.history.model.objects.filter(name=form['displayName'],history_type='~').order_by('-history_date')[1]
        occ.instance.save()
    except:
        print(form['displayName'])

  0%|          | 0/50 [00:00<?, ?it/s]

Camada Rolamento - Cbuq - Grad. C. - Com DOP
Camada Rolamento - Cbuq - Grad. 3 - Com DOP
Reciclagem Capa/Base Com Adição de 4% de Cimento
Sinalização Hor. Tinta P/ Pouco Trafego (Sin. Provisoria)


In [39]:
occ.__dict__

{'_state': <django.db.models.base.ModelState at 0x7efee731c7d0>,
 'uuid': UUID('a1224eb9-1e75-4c18-a02f-2a529ef043a4'),
 'name': 'IGG',
 'occurrence_kind': '7',
 'form_fields': {'id': 52,
  'name': 'IGG',
  'fields': [{'id': 1,
    'apiName': 'igg',
    'dataType': 'float',
    'displayName': 'IGG'}],
  'groups': [{'order': 1, 'members': [1], 'displayName': 'Parâmetro'}],
  'displayName': 'IGG',
  'measurementColumns': [{'key': 'igg',
    'logic': {'var': 'formData.igg'},
    'header': 'IGG'}]},
 'goal_formula': [],
 'created_at': datetime.datetime(2023, 4, 6, 13, 50, 58, 39598, tzinfo=<UTC>),
 'updated_at': datetime.datetime(2023, 9, 18, 19, 4, 32, 519631, tzinfo=<UTC>),
 'deadline': None,
 'active': True,
 'show_in_web_map': True,
 'show_in_app_map': False,
 'icon': '',
 'icon_size': 15,
 'color': '#005dffb3',
 'is_oae': False,
 'repetition': {},
 'monitoring_plan_id': None,
 'created_by_id': UUID('508a66f3-3f58-4c4e-bc9a-fbba6078c02d'),
 'previous_version_id': None,
 'history_id': U

In [49]:
old_forms = OccurrenceType.objects.filter(company=company).exclude(occurrence_kind='2')
old_forms.count()

297

In [30]:
obj=[x.name for x in old_forms]
counter=0

for a in [x['displayName'] for x in new_forms]:
    if a not in obj:
        print(a)
        counter+=1
        
print("\n")
print(counter)

LIMPEZA DE PLACA DE SINALIZAÇÃO


1


In [27]:
obj=[x.name for x in old_forms]

for a in obj:
    if "  " in a:
        print(a)

In [28]:
obj=[x['Classe'] for x in values]

for a in obj:
    if "  " in a:
        print(a)

In [24]:
for a in old_forms:
    if "  " in a.name:
        a.name=a.name.replace("  ", " ")
#         a.save()

In [17]:
obj=[x['Classe'] for x in values]
counter=0

for a in [x.name for x in old_forms]:
    if a not in obj:
        print(a)
        counter+=1
        
print("\n")
print(counter)

Relocação (solo/aérea) de Sinalização vertical
Limpeza de placa de sinalização
Remoção de placa de sinalização 
Limpeza (solo/aérea) de Sinalização vertical
Remoção (solo/aérea) de Sinalização vertical
Implantação (solo/aérea) de Sinalização vertical
Reparo (solo/aérea) de Sinalização vertical
Roçada  (Completa)


8


In [20]:
for form in new_forms:
    if form['displayName'] not in [a['Classe'] for a in values]:
        print(form['displayName'])
    else:
        row = next(a for a in values if a['Classe'] == form['displayName'])

{'Cor': None, 'Classe': 'Junta de dilatação termoelástica asfáltica para OAEs', 'id': 34, 'Prazo': None, 'Natureza': 'Juntas de Dilatação', 'id_Natureza': 13}
{'Cor': None, 'Classe': 'Superestrutura - Diamante', 'id': 12, 'Prazo': None, 'Natureza': 'Diamante', 'id_Natureza': 4}
{'Cor': None, 'Classe': 'Sinalização e Elementos de Proteção e Segurança - Pista Existente', 'id': 90, 'Prazo': None, 'Natureza': 'Sinalização e Elementos de Proteção e Segurança', 'id_Natureza': 31}
{'Cor': None, 'Classe': 'Infraestrutura - Diamante', 'id': 7, 'Prazo': None, 'Natureza': 'Diamante', 'id_Natureza': 4}
{'Cor': None, 'Classe': 'Armação de Pilar e/ou Viga', 'id': 24, 'Prazo': None, 'Natureza': 'Estruturas', 'id_Natureza': 9}
{'Cor': None, 'Classe': 'Armação em aço CA-50 - Aplicação', 'id': 67, 'Prazo': None, 'Natureza': 'Reforço da Fundação - Estacas', 'id_Natureza': 25}
{'Cor': None, 'Classe': 'CBUQ', 'id': 51, 'Prazo': None, 'Natureza': 'Pavimentação - Duplicação', 'id_Natureza': 19}
{'Cor': None,

In [21]:
row = next(a for a in values if a['Classe'] == form['displayName'])
print(row)

{'Cor': None, 'Classe': 'Dreno longitudinal de pavimento H = 1,00 m - com geocomposto drenante', 'id': 17, 'Prazo': None, 'Natureza': 'Drenagem', 'id_Natureza': 6}


In [22]:
print(next(a for a in values if a['Classe'] == form['displayName']))

{'Cor': None, 'Classe': 'Dreno longitudinal de pavimento H = 1,00 m - com geocomposto drenante', 'id': 17, 'Prazo': None, 'Natureza': 'Drenagem', 'id_Natureza': 6}
